In [ ]:
# @title Model training and saving

def train_model(save_dir, epochs = 500, train_size = 128, patience = 0):
    """
    Trains a YOLOv5 model on the specified dataset and saves the trained model
    to the specified directory.

    Parameters:
    -----------
    save_dir : str
        The directory where the trained model and output files will be saved.

    epochs : int, optional
        The number of training epochs. Default is 500.

    train_size : int, optional
        The size (width and height) of the input images used for training.
        Default is 128.

    patience : int, optional
        The number of epochs with no improvement after which training will be
        stopped early. Set to 0 to disable early stopping. Default is 0.

    Returns:
    --------
    None
    """

    model = YOLO("yolov5nu.pt")

    model.train(data=f"dataset/dataset.yaml",
                epochs=epochs,
                imgsz=train_size,
                batch=1, name="Telefinder_Model",
                patience=patience,
                project=f"Outputs/{save_dir}")

    model.save(f"Outputs/{save_dir}/Telefinder_Model.pt")

In [ ]:
# @title Bounding Box Drawing

def draw_bounding_boxes(save_dir, file_name, detections, encode_type):
    """
    Draws bounding boxes on an image based on the detection results and saves
    the annotated image.

    Parameters:
    -----------
    save_dir : str
        The directory where the annotated image will be saved.

    file_name : str
        The name of the image file (without extension) on which bounding boxes
        will be drawn.

    detections : list of tuples
        A list of detection results, where each detection is a tuple containing
        (x1, y1, x2, y2, confidence, class_id).

    encode_type : str
        The type of encoding used for the image (e.g., "Height", "Height
        Difference", "Point Count").

    Returns:
    --------
    numpy.ndarray
        The annotated image as a NumPy array in RGB format.
    """

    image_path = (
        f"Processed images/{encode_type} Encoded"
        f"/{file_name}.png"
    )
    output_image = cv2.imread(image_path)

    class_colors = {
        0 : [0, 255, 255],
        1 : [255, 102, 153]
    }

    for detection in detections:
        x1, y1, x2, y2, confidence, class_id = detection
        class_id = int(class_id)

        # Bounding box
        cv2.rectangle(output_image, (x1, y1),
                      (x2, y2), class_colors[class_id], 2)

        # Label and confidence score
        label = f"{CLASSES[class_id]}: {confidence:.2f}%"
        cv2.putText(output_image, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, class_colors[class_id], 1)

    # Convert image from cv2 to pil format
    output_image_rgb = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
    output_image_pil = Image.fromarray(output_image_rgb)

    save_path = (
        f"Outputs/{save_dir}/Labeled Images/"
        f"[{encode_type}]{file_name}.png"
    )

    output_image_pil.save(save_path)

    return output_image_rgb

In [ ]:
# @title Coordinate Dump
def coord_dump(save_dir, detections, classes):
    """
    Writes detection information, including coordinates, confidence scores,
    image filenames, and class labels, to a text file.

    Parameters:
    -----------
    save_dir : str
        The directory where the detection information will be saved.

    detections : list of tuples
        A list of detection results, where each detection is a tuple containing
        (center_x, center_y, confidence, image_file, class_id).

    classes : dict
        A dictionary mapping class IDs to class names.

    Returns:
    --------
    None
    """

    with open(f"Outputs/{save_dir}/detections.txt", 'w') as f:
        f.write("-----------------------------------------------------------\n")
        f.write("|         Info dump on successful detections\n")
        f.write("|  < X >   < Y >   < Confidence >  < Image >   < Class >  |\n")
        f.write("-----------------------------------------------------------\n")
        for detection in detections:
            center_x, center_y, confidence, image_file, class_id = detection

            f.write(f"< {center_x}  {center_y}  {confidence:.2f}"
                    f"  {image_file}  {classes[class_id]} >\n")

In [ ]:
# @title Detect

def run_detections(save_dir, classes, file_list,
                   encode_type = "Height", seg_size = 512, overlap = 0.1):
    """
    Runs object detection on a list of images by segmenting each image into
    smaller parts, applying the model to each segment, and drawing bounding
    boxes around detected objects. The function also saves the detection
    coordinates to a file.

    Parameters:
    -----------
    save_dir : str
        The directory where the results, including annotated images and
        detection information, will be saved.

    classes : dict
        A dictionary mapping class IDs to class names.

    file_list : list of str
        A list of image filenames (without extensions) to run detections on.

    encode_type : str, optional
        The type of encoding used for the images (e.g., "Height", "Height
        Difference", "Point Count"). Default is "Height".

    seg_size : int, optional
        The size (width and height) of the segments into which each image will
        be divided for detection. Default is 512.

    overlap : float, optional
        The overlap fraction between consecutive segments. Default is 0.1.

    Returns:
    --------
    tuple
        A tuple containing two lists:
        - total_images: A list of images with drawn bounding boxes (NumPy
          arrays).
        - file_list: The list of image filenames that were processed.
    """

    model = YOLO(f"Outputs/{save_dir}/Telefinder_Model.pt")

    total_detections = []
    total_images = []

    for image_file in file_list:

        image = cv2.imread(
            f"Processed images/{encode_type} Encoded/{image_file}.png")

        step_size = int(seg_size * (1 - overlap))
        height, width, _ = image.shape
        detections = []

        for y in range(0, height - seg_size + 1, step_size):
            for x in range(0, width - seg_size + 1, step_size):
                segment = image[y:y + seg_size, x:x + seg_size]

                # Run detection on the segment
                results_list = model(segment)

                for results in results_list:
                    if results.boxes is not None:
                        for box in results.boxes:
                            x1, y1, x2, y2 = map(int, box.xyxy[0])
                            confidence = box.conf[0] * 100
                            class_id = int(box.cls[0])

                            x1 += x
                            y1 += y
                            x2 += x
                            y2 += y

                            center_x = int((x1 + x2) / 2)
                            center_y = int((y1 + y2) / 2)

                            detections.append([x1, y1, x2, y2,
                                               confidence, class_id])
                            total_detections.append([center_x,
                                                     center_y,
                                                     confidence,
                                                     image_file,
                                                     class_id])

        total_images.append(draw_bounding_boxes(save_dir,
                                                image_file,
                                                detections,
                                                encode_type))
    coord_dump(save_dir, total_detections, classes)

    return total_images, file_list